In [161]:
import numpy as np
from typing import Union, List, Callable

# Lagrange Interpolation
If $x_0,x_1,\cdots,x_n$ are $n+1$ distinct numbers and $f$ is a function whose values are given at these numbers, then a unique polynomial $P(x)$ of degree at most $n$ exists with
$$f(x_k)=P(x_k),\,\,\,\text{for each }k=0,1,\cdots,n$$
This polynomial is given by
$$P(x)=f(x_0)L_{n, 0}(x)+\cdots+f(x_n)L_{n, n}(x)=\sum^n_{k=0}f(x_k)L_{n, k}(x)$$
where for each $k=0,1,\cdots,n$
$$L_{n, k}(x)=\frac{(x-x_0)(x-x_1)\cdots(x-x_{k-1})(x-x_{k+1})\cdots(x-x_n)}{(x_k-x_0)(x_k-x_1)\cdots(x_k-x_{k-1})(x_k-x_{k+1})\cdots(x_k-x_n)}=\prod^n_{\substack{i=0\\i\neq k}}\frac{(x-x_i)}{(x_k-x_i)}$$

In [162]:
def LagrangeInterpolation(func:Union[Callable[[float], float], np.ndarray, List], x_:Union[np.ndarray, List])->Callable[[float], float]:
    def interpolation(x:float)->float:
        call_flag = 1 and callable(func)
        appro = 0
        for i, x_i in enumerate(x_):
            l_i = 1
            for k, x_k in enumerate(x_):
                if k != i:
                    l_i *= (x - x_k) / (x_i - x_k)
            appro += func(x_i) * l_i if call_flag else func[i] * l_i
        return appro
    return interpolation

# Neville's Method
$$ Q_{i, j}=\frac{(x-x_{i-j})Q_{i, j-1}-(x-x_i)Q_{i-1, j-1}}{x_i-x_{i-j}}$$
where $P(x)=Q_{n, n}$

In [163]:
def Neville(func:Union[Callable[[float], float], np.array, List], x_:Union[np.ndarray, List])->Callable[[float], float]:
    def interpolation(x:float)->float:
        call_flag = 1 and callable(func)
        len_x = sum(1 for _ in x_)
        Q = np.zeros((len_x, len_x))
        for idx in range(len_x):
            Q[idx][0] = func(x_[idx]) if call_flag else func[idx]
        for idx in range(1, len_x):
            for jdx in range(1, idx + 1):
                Q[idx][jdx] = ((x - x_[idx - jdx]) * Q[idx][jdx - 1] - (x - x_[idx]) * Q[idx - 1][jdx - 1]) / (x_[idx] - x_[idx - jdx])
        return Q[-1][-1]
    return interpolation

# Newton's Dividend-Difference Formula

# Hermite Interpolation

In [164]:
def Hermite(func:Union[Callable[[float], float], np.array, List], dfunc:Union[Callable, np.array, List], x_:Union[np.ndarray, List])->Callable[[float], float]:
    def interpolation(x:float)->float:
        f_call = 1 and callable(func)
        df_call = 1 and callable(dfunc)
        len_x = sum(1 for i in x_)
        
        z = np.zeros(2 * len_x)
        Q = np.zeros((2 * len_x, 2 * len_x))

        for idx in range(len_x):
            z[2 * idx] = x_[idx]
            z[2 * idx + 1] = x_[idx]
            Q[2 * idx, 0] = func(x_[idx]) if f_call else func[idx]
            Q[2 * idx + 1, 0] = func(x_[idx]) if f_call else func[idx]
            Q[2 * idx + 1, 1] = dfunc(x_[idx]) if df_call else dfunc[idx]
            if idx:
                Q[2 * idx, 1] = (Q[2 * idx, 0] - Q[2 * idx - 1, 0]) / (z[2 * idx] - z[2 * idx - 1])
        for idx in range(2, 2 * len_x):
            for jdx in range(2, idx + 1):
                Q[idx][jdx] = (Q[idx][jdx - 1] -  Q[idx - 1][jdx - 1]) / (z[idx] - z[idx - jdx])
        print(z)
        print(Q)
        return Q
    return interpolation

In [165]:
i = Hermite(
    [0.6200860, 0.4554022, 0.2818186],
    [-0.5220232, -0.5698959, -0.5811571],
    [1.3, 1.6, 1.9]
)
for j in i(0):
    print(j)

[1.3 1.3 1.6 1.6 1.9 1.9]
[[ 0.620086    0.          0.          0.          0.          0.        ]
 [ 0.620086   -0.5220232   0.          0.          0.          0.        ]
 [ 0.4554022  -0.548946   -0.08974267  0.          0.          0.        ]
 [ 0.4554022  -0.5698959  -0.069833    0.06636556  0.          0.        ]
 [ 0.2818186  -0.578612   -0.02905367  0.06796556  0.00266667  0.        ]
 [ 0.2818186  -0.5811571  -0.00848367  0.06856667  0.00100185 -0.00277469]]
[0.620086 0.       0.       0.       0.       0.      ]
[ 0.620086  -0.5220232  0.         0.         0.         0.       ]
[ 0.4554022  -0.548946   -0.08974267  0.          0.          0.        ]
[ 0.4554022  -0.5698959  -0.069833    0.06636556  0.          0.        ]
[ 0.2818186  -0.578612   -0.02905367  0.06796556  0.00266667  0.        ]
[ 0.2818186  -0.5811571  -0.00848367  0.06856667  0.00100185 -0.00277469]
